In [1]:
from __future__ import division
from IPython.core.debugger import Pdb
import pickle
import ast
import random
import numpy as np
import enchant
import matplotlib.pyplot as plt
from nltk import bigrams,ConditionalFreqDist,FreqDist,pos_tag,pos_tag_sents
from gensim import parsing, matutils, interfaces, corpora, models, similarities, summarization
from gensim.utils import lemmatize
from nltk import collocations, association, text, tree
from gensim.corpora.mmcorpus import MmCorpus
from gensim.matutils import corpus2csc
from gensim.similarities.docsim import SparseMatrixSimilarity
from reader import Json100CorpusReader as js100
from reader import BlogCorpusReader
import itertools
from collections import Counter
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
#from sklearn.naive_bayes import MultinomialNB,BernoulliNB
#from sklearn.svm import SVC, LinearSVC
#from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
#from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
#from sklearn.tree import DecisionTreeClassifier 
#from sklearn.learning_curve import learning_curve
import re

from lxml import etree
from StringIO import StringIO
from os import listdir
from os.path import join as pathjoin

import json
import pandas as pd

from collections import defaultdict

from nltk.grammar import DependencyGrammar
from nltk.parse import (
    DependencyGraph, ProjectiveDependencyParser, NonprojectiveDependencyParser)
from nltk.parse.malt import MaltParser as MaltParser

from nltk.corpus import dependency_treebank as dt
from nltk.corpus import treebank_raw
from nltk.corpus import treebank
import os
from pickle import load
from nltk.parse import stanford
import operator
from nltk.corpus.util import LazyCorpusLoader

from glob import iglob
from StringIO import StringIO
from os import listdir
from os.path import join as pathjoin
from nltk.tag.stanford import StanfordPOSTagger
import re

tla = ['abo', 'sub', 'apa']
NAME = os.environ.get('NAME') or 'dmoz'
wdir = os.path.expanduser('~/scrapy')
os.environ['CLASSPATH'] = pathjoin(wdir, 'lib')
odir = pathjoin(wdir, NAME)
os.chdir(odir)

craigcr = js100(odir, r'marker1')

In [2]:
def make_dict(crlist):
    stoplist = set('for a of the and to in'.split())
    stoplist = set()
    class nouner(object):
        def __init__(self, cr):
            self.cr = cr
        def __iter__(self):
            for doc in self.cr.sents():
                yield doc
                # for t in list(parser.parse(sent)):
                #     for p in t.productions():
                #         if p.is_lexical():
                #             if p.lhs() == 'NN':
                #                 yield 'NN'
                #             else:
                #                 print p.rhs()
                #                 yield p.rhs()
    # corpora.Dictionary is a static method of gensim.corpora
    # it establishes the base of operations numbering the vocab,
    # and you feed it strings such as doc2bow(feedit) produces a sparse vector.
    dictionary = corpora.Dictionary(list(itertools.chain.from_iterable([list(nouner(cr)) for cr in crlist])))
    checker = enchant.Dict("en_US")
    stop_ids = [dictionary.token2id[stopword] for stopword in stoplist 
                if stopword in dictionary.token2id]
    once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq == 1]
    mispelled_ids = [dictionary.token2id[word] for word in dictionary.values() if not checker.check(word)]
    dictionary.filter_tokens(stop_ids + once_ids + mispelled_ids)
    
    myset = set(['WRB', 'WP$', 'WP', 'WDT', 'TO', 'RP', 'PRP$', 'PRP', 'PDT', 'MD', 'IN', 'EX', 'DT', 'CC', 'UH', ])
    myset = set(['WRB', 'WP$', 'WP', 'WDT', 'TO', 'RP', 'PRP$', 'PRP', 'PDT', 'MD', 'IN', 'EX', 'DT', 'CC', 'RB', 'RBR', 'RBS', 'UH', 'NNP', 'NNS',  ])
    myset = set(['WRB', 'WP$', 'WP', 'WDT', 'TO', 'RP', 'PRP$', 'PRP', 'PDT', 'MD', 'IN', 'EX', 'DT', 'CC', 'RB', 'RBR', 'RBS', 'UH', 'NNP', 'NNS',  ])
    myset = set(['WRB', 'WP$', 'WP', 'WDT', 'TO', 'RP', 'PRP$', 'PRP', 'PDT', 'MD', 'IN', 'EX', 'DT', 'CC', 'UH', 'RB', 'RBR', 'RBS', 'NN', 'NNP', 'NNS', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBZ' ])
    contextual_ids = [dictionary.token2id[k[0][0]] for k in pos_tag_sents([[w] for w in dictionary.values()]) if k[0][1] not in myset]
    
#    dictionary.filter_tokens(contextual_ids)
    dictionary.compactify()
    return dictionary

In [3]:
def CorpusDedupe(cr, dict):
    corpus = [dict.doc2bow(doc[:30]) for doc in list(cr)]
    all = set(range(0,len(corpus)))
    index = SparseMatrixSimilarity(corpus,num_features=len(dict.keys()))
    marked = set()
    for s, similarities in enumerate(index):
        if s in marked:
            continue
        for i, item in enumerate(similarities[s+1:]):
            if item > .99:
		marked.add(s)
                marked.add(s+1+i)

    # you recover the original indexing through marked
    corpus = [dict.doc2bow(doc) for doc in list(cr)]
    for m in sorted(marked, reverse=True):
        del corpus[m]
    return (all.difference(marked), corpus)


In [4]:
# establish vocab numbering
dict = make_dict([craigcr])
# corpus = collection of bow sparse vectors
(marked, corpus) = CorpusDedupe(craigcr, dict)


In [5]:
from math import radians, sin, cos, sqrt, asin
def haversine(lat1, lon1, lat2, lon2):
    R = 6372.8 # Earth radius in kilometers
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
 
    return R * c

def within(coords):
    if coords[0] is None or coords[1] is None:
	return False
    if NAME == "dmoz":
	return float(coords[0]) < 40.796126
    # que
    elif NAME == "que":
	km = haversine(40.748688, -73.93798, float(coords[0]), float(coords[1]))
	return km < 3
    # sf
    elif NAME == "sfc":
	km = haversine(37.779076, -122.397501, float(coords[0]), float(coords[1]))
	return km < 1.5
    # berkeley
    elif NAME == "eby":
	km = haversine(37.871454, -122.298115, float(coords[0]), float(coords[1]))
	return km < 3
    # milbrae
    #    km = haversine(37.600122, -122.386914, float(coords[0]), float(coords[1]))
    # daly city
    #    km2 = haversine(37.687915, -122.472452, float(coords[0]), float(coords[1]))
    return True

coords = craigcr.coords()
outside = set()
for ci in marked:
    
    if not within(coords[ci]):
	outside.add(ci)
marked = marked.difference(outside)


In [6]:
coords = craigcr.coords()
sameloc = set()
for ci in marked:
    if coords[ci][0] is None:
	    continue
    if ci in sameloc:
	    continue
    # for j,c2 in enumerate(list(marked[type])[i+1:]):
    # 	if coords[c2] == coords[c]:
    # 	    sameloc.add(i)
    # 	    sameloc.add(i+1+j)
    for cj,c2 in enumerate(coords):
	    if ci == cj:
		continue
	    if c2 == coords[ci]:
		sameloc.add(ci)
		sameloc.add(cj)
		break
# same lat and long is okay ... comment below
# but leasebreak piles into same coord
# marked[type] = marked[type].difference(sameloc)
len(marked)


20

In [7]:
def filter(d, listedby):
    ok = bool(listedby) # if sublet (not listedby) then default to suspicious
    for w in d:
#	if w in set(['broker', 'fee', 'fully', 'brokerage', 'leasebreak', 'female', 'rentsfnow', 'hivesf', 'heritage', 'realty', 'application', 'yr', 'year', '12', ]):
#	    return False
	if re.compile("!!+|leasebreak|dalia|greystar", re.IGNORECASE).search(w):
	    return False
	if re.compile("^(i|me|mine|our|he|she|they|their|we|my|his|her)$", re.IGNORECASE).search(w):
	    ok = True
    return ok

In [8]:
from collections import Counter
listedby = craigcr.listedby()
filtListedBy = set()
for pair in Counter(listedby).iteritems():
    if not pair[0]:
	filtListedBy.add(pair[0])
    elif pair[1] == 1:
	if not re.compile("no fee|contact|apartments|apts|for all|llc|to view|===+|----+|\*\*\*+|\.\.\.+|xxxx+|best|mark a|marc a|rentals|real|estate", re.IGNORECASE).search(pair[0]):
	    if len(pair[0].split()) < 5:
		filtListedBy.add(pair[0])

In [9]:
mark = {}
for type in tla:
    try:
	with open(pathjoin(odir, 'marker0'), 'r') as f0:
	    mark[type] = max(['0'] + [re.findall(r"{0}/(\d+).html".format(type), line)[-1] for line in f0.readlines() if re.findall(r"{0}/(\d+).html".format(type), line)])
    except (IOError, IndexError) as e:
	mark[type] = '0'


In [10]:
links = craigcr.links()
listedby = craigcr.listedby()
lst = ['%s %s %s' % (' '.join(d[0:50]), links[i], listedby[i] if listedby[i] else "Actual Person?") for (i, d) in enumerate(craigcr.docs()) if len(d) > 10  and i in set(marked) and filter(d, listedby[i]) and listedby[i] in filtListedBy]


In [26]:
with open(pathjoin(odir, 'digest'), 'w+') as f1:
    for type in tla:
        for listing in lst:
	    try:
                id = re.findall(r"{0}/(?:[^/]+/)*?(\d+).html".format(type), listing)[-1]
		if id > mark[type]:
		    f1.write(listing.encode('ascii', 'ignore') + '\n\n')
	    except IndexError as e:
		pass

In [137]:
os.rename(pathjoin(odir, 'marker1'), pathjoin(odir, 'marker0'))